In [1]:
import numpy as np
import scipy
from scipy.spatial.distance import cdist
%matplotlib qt
import matplotlib.pyplot as plt

In [2]:
def ker(x1, x2):
    sq_norm = -0.5 * (x1-x2)**2
    return np.exp(sq_norm)

In [ ]:
# Visualise samples from a GP (noise-free)

# Figure handles
f_prior, a_prior = plt.subplots(1, 1)
f_post, a_post = plt.subplots(1, 1)

# Plotting/sampling parameters
n_fn = 5
n_pts = 100
downsample_factor = 20


# Generate Samples from the Prior 
x_prior_sample = np.expand_dims(np.sort(np.linspace(-4, 4, n_pts)), 1)
K = ker(x_prior_sample, x_prior_sample.T) + 1.e-8*np.eye(n_pts)  # Kernel of data points
K_inv = np.linalg.inv(K)
y_prior_samples = np.random.multivariate_normal(mean=np.zeros(n_pts), cov=K, size=n_fn)

# Plot Prior
for y in y_prior_samples:
    a_prior.plot(x_prior_sample, y)

prior_stdev = np.sqrt(K.diagonal())
a_prior.plot(x_prior_sample, np.zeros_like(x_prior_sample), 'k')
a_prior.fill(np.concatenate([x_prior_sample, x_prior_sample[::-1]]),
             np.concatenate([2.*prior_stdev, -2.*prior_stdev[::-1]]),
            alpha=0.25, color='lightblue')
a_prior.plot(x_prior_sample, 2.* prior_stdev, '--', color='grey')
a_prior.plot(x_prior_sample, -2.* prior_stdev, '--', color='grey')

# Sample a few points from the Prior
y_prior_samples = y_prior_samples[:, ::downsample_factor]
x_prior_sample  = x_prior_sample[::downsample_factor,:]
K = ker(x_prior_sample, x_prior_sample.T) + 1.e-8*np.eye(x_prior_sample.size)  # Kernel of data points
K_inv = np.linalg.inv(K)

# Generate Posterior
y_prior_samp  = np.expand_dims(y_prior_samples[0], 1)
x_post_sample = np.expand_dims(np.sort(np.linspace(-4, 4, 100)), 1)
K_star        = ker(x_prior_sample, x_post_sample.T)
K_star_star   = ker(x_post_sample, x_post_sample.T)
post_cov      = K_star_star - K_star.T.dot(K_inv.dot(K_star))
post_mean     = K_star.T.dot(K_inv).dot(y_prior_samp).flatten()

y_post_samples = np.random.multivariate_normal(mean=post_mean, cov=post_cov, size=n_fn)

# Plot Posterior
for y in y_post_samples:
    a_post.plot(x_post_sample, y)
a_post.plot(x_prior_sample, y_prior_samp, 'k*')

post_stdev = np.sqrt(post_cov.diagonal())
a_post.plot(x_post_sample, post_mean, 'k')
a_post.plot(x_post_sample, post_mean + 2.* post_stdev, '--', color='grey')
a_post.plot(x_post_sample, post_mean - 2.* post_stdev, '--', color='grey')
a_post.fill(np.concatenate([x_post_sample, 
                            x_post_sample[::-1]]),
            np.concatenate([post_mean + 2.*post_stdev, 
                            (post_mean - 2.*post_stdev)[::-1]]),
            alpha=0.25, color='lightblue')

for a in [a_prior, a_post]:
    a.set_xlim(-4, 4)
    a.set_ylim(-2, 2)
    a.grid()

a_prior.set_xlabel('x')
a_post.set_xlabel('x')
a_prior.set_ylabel('g')
a_prior.set_title('Samples from GP Prior')
a_post.set_title('Samples from GP Posterior')
plt.show()

In [11]:
# Visualise samples from a GP (with noise)

# Figure handles
f_prior, a_prior = plt.subplots(1, 1)
f_post, a_post   = plt.subplots(1, 1)

# Plotting/sampling parameters
n_fn = 5
n_pts = 100
downsample_factor = 20
noise_var = 0.1**2

# Generate Samples from the Prior 
x_prior_sample = np.expand_dims(np.sort(np.linspace(-4, 4, n_pts)), 1)
K = ker(x_prior_sample, x_prior_sample.T)
K_inv = np.linalg.inv(K)
y_prior_samples = np.random.multivariate_normal(mean=np.zeros(n_pts), cov=K, size=n_fn)

# Plot Prior
for y in y_prior_samples:
    a_prior.plot(x_prior_sample, y)

prior_stdev = np.sqrt(K.diagonal())
a_prior.plot(x_prior_sample, np.zeros_like(x_prior_sample), 'k')
a_prior.fill(np.concatenate([x_prior_sample, x_prior_sample[::-1]]),
             np.concatenate([2.*prior_stdev, -2.*prior_stdev[::-1]]),
            alpha=0.25, color='lightblue')
a_prior.plot(x_prior_sample, 2.* prior_stdev, '--', color='grey')
a_prior.plot(x_prior_sample, -2.* prior_stdev, '--', color='grey')

# Sample a few points from the Prior
y_prior_samples = y_prior_samples[:, ::downsample_factor]
x_prior_sample  = x_prior_sample[::downsample_factor,:]
K = ker(x_prior_sample, x_prior_sample.T) + noise_var*np.eye(x_prior_sample.size)  # Kernel of data points
K_inv = np.linalg.inv(K)

# Generate Posterior
y_prior_samp  = np.expand_dims(y_prior_samples[0], 1)
x_post_sample = np.expand_dims(np.sort(np.linspace(-4, 4, 100)), 1)
K_star        = ker(x_prior_sample, x_post_sample.T)
K_star_star   = ker(x_post_sample, x_post_sample.T)
post_cov      = K_star_star - K_star.T.dot(K_inv.dot(K_star))
post_mean     = K_star.T.dot(K_inv).dot(y_prior_samp).flatten()

y_post_samples = np.random.multivariate_normal(mean=post_mean, cov=post_cov, size=n_fn)

# Plot Posterior
for y in y_post_samples:
    a_post.plot(x_post_sample, y)
a_post.plot(x_prior_sample, y_prior_samp, 'k*')

post_stdev = np.sqrt(post_cov.diagonal())
a_post.plot(x_post_sample, post_mean, 'k')
a_post.plot(x_post_sample, post_mean + 2.* post_stdev, '--', color='grey')
a_post.plot(x_post_sample, post_mean - 2.* post_stdev, '--', color='grey')
a_post.fill(np.concatenate([x_post_sample, 
                            x_post_sample[::-1]]),
            np.concatenate([post_mean + 2.*post_stdev, 
                            (post_mean - 2.*post_stdev)[::-1]]),
            alpha=0.25, color='lightblue')

for a in [a_prior, a_post]:
    a.set_xlim(-4, 4)
    a.set_ylim(-2.5, 2.5)
    a.grid()

a_prior.set_xlabel('x')
a_post.set_xlabel('x')
a_prior.set_ylabel('g')
a_prior.set_title('Samples from GP Prior')
a_post.set_title('Samples from GP Posterior')
plt.show()

In [48]:
# Fit a GP with different hyperparameters 
class GP(object):
    
    def __init__(self, var_f, var_n, L):
        self.var_f = var_f
        self.var_n = var_n
        self.L     = L
        self.y     = None
        self.K_inv = None
    
    def fit(self, X, y):
        self.y = y
        self.X = X
        self.K = self.ker_hparams(X, X.T) + self.var_n*np.eye(y.size) 
        self.K_inv = np.linalg.inv(self.K)
        
    def pred_posterior(self, X_pred):
        K_star        = self.ker_hparams(self.X, X_pred.T)
        K_star_star   = self.ker_hparams(X_pred, X_pred.T) + self.var_n*np.eye(X_pred.size)
        post_cov      = K_star_star - K_star.T.dot(self.K_inv.dot(K_star))
        post_mean     = K_star.T.dot(self.K_inv).dot(self.y).flatten()
        return post_mean, post_cov.diagonal()

    def ker_hparams(self, x1, x2):
        sq_norm = -0.5 * ((x1-x2) / self.L)**2
        return self.var_f * np.exp(sq_norm)

for lengthscale, stdev_f, stdev_n in zip([1.0, 0.3, 3.0],
                                         [1.0, 0.5, 1.1],
                                         [0.1, 0.00005, 0.5]):
    plt.figure()
    m = GP(stdev_f**2, stdev_n**2, lengthscale)

    m.fit(x_prior_sample, y_prior_samp)
    mu_pred, var_pred = m.pred_posterior(x_post_sample)
    stdev_pred = np.sqrt(var_pred)

    l, = plt.plot(x_post_sample, mu_pred)
    plt.fill(np.concatenate([x_post_sample, 
                                x_post_sample[::-1]]),
                np.concatenate([mu_pred + 2.*stdev_pred, 
                                (mu_pred - 2.*stdev_pred)[::-1]]),
                alpha=0.25, color=l.get_color())
    plt.plot(x_prior_sample, y_prior_samp, 'k*')
    plt.title(r'(L,sig_f, sig_n) = ({0}, {1}, {2})'.format(lengthscale, stdev_f, stdev_n))
    plt.grid()
    plt.ylim(-2, 1)
    plt.xlim(-4, 4)
plt.show()

In [58]:
grid_pts = 50
n_pts = 5
n_fn = 5

x_prior_sample = np.expand_dims(np.sort(np.linspace(-7, 7, n_pts)), 1)
K = ker(x_prior_sample, x_prior_sample.T) + np.eye(n_pts)*0.5
K_inv = np.linalg.inv(K)
y_prior_samples = np.random.multivariate_normal(mean=np.zeros(n_pts), cov=K, size=n_fn)


for y_prior_samp in y_prior_samples:
    y_prior_samp.reshape(-1,1)
    
    plt.figure()
    likelihood = np.zeros([grid_pts, grid_pts])
    for i, lengthscale_pow in enumerate(np.linspace(-2, 1, grid_pts)):
        for j, var_n_pow in enumerate(np.linspace(-2, 1, grid_pts)):
            m.var_n = np.power(10, var_n_pow)
            m.L = np.power(10, lengthscale_pow)
            m.fit(x_prior_sample, y_prior_samp)
            likelihood[i,j] = y_prior_samp.T.dot(m.K.dot(y_prior_samp)) - np.log(np.linalg.det(m.K))
    plt.imshow(likelihood)
plt.show()